# Project 1 - Interpretability



 https://www.kaggle.com/code/jacobmillet/cs-go-round-winner-prediction-xgb-87-acc
 

## Imports

In [ ]:
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib as plt

## Dataset import and feature set

In [ ]:
dataset = pd.read_csv("./dataset/csgo_round_snapshots.csv")
features = dataset.columns

y = dataset['round_winner']
X = dataset.drop(['round_winner'], axis=1)

cols_grenade = 'grenade'
X = X.drop(X.columns[X.columns.str.contains(cols_grenade)], axis=1)

# Possibly lower the amount of columns by combining some


Index(['time_left', 'ct_score', 't_score', 'map', 'bomb_planted', 'ct_health',
       't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets',
       't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive',
       'ct_weapon_ak47', 't_weapon_ak47', 'ct_weapon_aug', 't_weapon_aug',
       'ct_weapon_awp', 't_weapon_awp', 'ct_weapon_bizon', 't_weapon_bizon',
       'ct_weapon_cz75auto', 't_weapon_cz75auto', 'ct_weapon_elite',
       't_weapon_elite', 'ct_weapon_famas', 't_weapon_famas',
       'ct_weapon_g3sg1', 't_weapon_g3sg1', 'ct_weapon_galilar',
       't_weapon_galilar', 'ct_weapon_glock', 't_weapon_glock',
       'ct_weapon_m249', 't_weapon_m249', 'ct_weapon_m4a1s', 't_weapon_m4a1s',
       'ct_weapon_m4a4', 't_weapon_m4a4', 'ct_weapon_mac10', 't_weapon_mac10',
       'ct_weapon_mag7', 't_weapon_mag7', 'ct_weapon_mp5sd', 't_weapon_mp5sd',
       'ct_weapon_mp7', 't_weapon_mp7', 'ct_weapon_mp9', 't_weapon_mp9',
       'ct_weapon_negev', 't_weapon_negev',